In [1]:
# %pip install requests pandas pyyaml tqdm

  Using cached pandas-2.2.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached numpy-2.1.1-cp311-cp311-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp311-cp311-macosx_11_0_arm64.whl (11.3 MB)
Using cached tqdm-4.66.5-py3-none-any.whl (78 kB)
Using cached numpy-2.1.1-cp311-cp311-macosx_14_0_arm64.whl (5.4 MB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import pandas as pd
import yaml
import time
from tqdm import tqdm

# Load API key from config file
with open('config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)
    api_key = config['api_key']

# Read LinkedIn URLs from CSV
df = pd.read_csv('data/Customer Discovery Excel 1.csv')
linkedin_urls = df['LinkedIn URL'].dropna().tolist()

# Initialize list to store scraped data
scraped_data = []

# Scrapetable API endpoint
url = 'https://api.scrapetable.com/linkedin/profiles'

# Scrape data in batches of 50
for i in tqdm(range(0, len(linkedin_urls), 50)):
    batch = linkedin_urls[i:i+50]
    
    payload = {
        'links': batch,
        'flatten': False
    }
    
    headers = {
        'Content-Type': 'application/json',
        'api-key': api_key
    }
    
    response = requests.post(url, json=payload, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        scraped_data.extend(data)
    else:
        print(f"Error for batch starting at index {i}: {response.status_code}")
    
    # Sleep for 2 minutes after each batch (except the last one)
    if i + 50 < len(linkedin_urls):
        time.sleep(120)

# Convert scraped data to DataFrame
result_df = pd.DataFrame(scraped_data)

# Export scraped data to CSV
result_df.to_csv('data/scraped_data.csv', index=False)

print("Scraping completed. Data saved to 'data/scraped_data.csv'")

100%|██████████| 22/22 [44:31<00:00, 121.42s/it]

Scraping completed. Data saved to 'data/scraped_data.csv'
